#### Some preprocessing on the AMPds2 Dataset

In [1]:
import pandas as pd
import numpy as np 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [2]:
weather = pd.read_csv(os.path.join('data','Climate_HourlyWeather.csv'), parse_dates=['Date/Time'])
data =pd.read_csv(os.path.join('data', 'Electricity_P.csv')) 

In [3]:
weather.shape, data.shape

((17520, 25), (1051200, 24))

**Power data** : 
* resampling to 30 minutes
* keeping one metric: WHE, the total power consumption

In [4]:
data.head()

,UNIX_TS,WHE,RSE,GRE,MHE,B1E,BME,CWE,DWE,EQE,...,WOE,B2E,CDE,DNE,EBE,FGE,HTE,OUE,TVE,UNE
0,1333263600,918,43,2,873,0,352,0,0,41,...,0,58,0,30,0,0,4,0,37,138
1,1333263660,913,43,0,870,0,354,0,0,35,...,0,58,0,30,0,0,4,0,37,138
2,1333263720,872,43,2,827,0,354,0,0,41,...,0,8,0,30,0,0,4,0,37,138
3,1333263780,872,43,2,827,0,354,0,0,40,...,0,7,0,30,0,0,4,0,37,142
4,1333263840,772,41,2,729,0,352,0,0,40,...,0,8,0,31,0,0,4,0,37,45


In [5]:
elec = data[['WHE', 'UNIX_TS']].rename(columns = {'WHE': 'load', 'UNIX_TS':'date'}).set_index('date')

In [6]:
elec.head()

,load
date,
1333263600,918
1333263660,913
1333263720,872
1333263780,872
1333263840,772


In [7]:
elec.index = pd.to_datetime(elec.index, unit="s")

In [8]:
elec.head()

,load
date,
2012-04-01 07:00:00,918
2012-04-01 07:01:00,913
2012-04-01 07:02:00,872
2012-04-01 07:03:00,872
2012-04-01 07:04:00,772


In [9]:
elec_res = elec.resample('30T').mean()

In [10]:
elec_res.head()

,load
date,
2012-04-01 07:00:00,1947.400000
2012-04-01 07:30:00,1269.766667
2012-04-01 08:00:00,1645.300000
2012-04-01 08:30:00,845.666667
2012-04-01 09:00:00,1584.966667


In [11]:
elec_res.shape

(35040, 1)

One version with timestamps

In [13]:
elec_kit = elec_res.copy(deep=True)
elec_kit.index = (elec_kit.index.astype(int)/1e9).astype(int)

In [14]:
elec_states = elec_kit.drop('load',1)

In [40]:
elec_kit.to_csv('data/ampds_power_30T.csv')
elec_states.to_csv('data/ampds_power_30T_states.csv')

One version with timezoned ISO dates

In [12]:
elec_iso = elec_res.copy(deep=True)
elec_iso.index = elec_iso.index.tz_localize('UTC').tz_convert('Canada/Pacific')
elec_iso.index = elec_iso.index.map(lambda x : x.isoformat())
elec_states_iso = elec_iso.drop('load',1)

In [13]:
elec_iso.head()

,load
date,
2012-04-01T00:00:00-07:00,1947.400000
2012-04-01T00:30:00-07:00,1269.766667
2012-04-01T01:00:00-07:00,1645.300000
2012-04-01T01:30:00-07:00,845.666667
2012-04-01T02:00:00-07:00,1584.966667


In [14]:
elec_iso.to_csv('data/ampds_power_30T_iso.csv')
elec_states_iso.to_csv('data/ampds_power_30T_states_iso.csv')

**Weather** : 

* make cache for energy kit
* resample in csv for benchmark models

In [5]:
weather.head()

,Date/Time,Year,Month,Day,Time,Data Quality,Temp (C),Temp Flag,Dew Point Temp (C),Dew Point Temp Flag,...,Wind Spd Flag,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather
0,2012-04-01 00:00:00,2012,4,1,00:00,NaN,4.3,NaN,2.3,NaN,...,NaN,12.9,NaN,99.88,NaN,NaN,NaN,NaN,NaN,Rain
1,2012-04-01 01:00:00,2012,4,1,01:00,NaN,4.4,NaN,2.2,NaN,...,NaN,8.0,NaN,99.94,NaN,NaN,NaN,NaN,NaN,Rain - Fog
2,2012-04-01 02:00:00,2012,4,1,02:00,NaN,3.8,NaN,1.6,NaN,...,NaN,6.4,NaN,100.00,NaN,NaN,NaN,NaN,NaN,Rain - Fog
3,2012-04-01 03:00:00,2012,4,1,03:00,NaN,2.8,NaN,0.7,NaN,...,NaN,8.0,NaN,100.10,NaN,NaN,NaN,NaN,NaN,Rain - Fog
4,2012-04-01 04:00:00,2012,4,1,04:00,NaN,2.4,NaN,0.4,NaN,...,NaN,9.7,NaN,100.23,NaN,NaN,NaN,NaN,NaN,Moderate Rain


In [6]:
weather = weather.drop(['Year', 'Month', 'Day', 'Time', 'Data Quality'],1)

In [7]:
weather.head()

,Date/Time,Temp (C),Temp Flag,Dew Point Temp (C),Dew Point Temp Flag,Rel Hum (%),Rel Hum Flag,Wind Dir (10s deg),Wind Dir Flag,Wind Spd (km/h),Wind Spd Flag,Visibility (km),Visibility Flag,Stn Press (kPa),Stn Press Flag,Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag,Weather
0,2012-04-01 00:00:00,4.3,NaN,2.3,NaN,87.0,NaN,30.0,NaN,28.0,NaN,12.9,NaN,99.88,NaN,NaN,NaN,NaN,NaN,Rain
1,2012-04-01 01:00:00,4.4,NaN,2.2,NaN,86.0,NaN,30.0,NaN,17.0,NaN,8.0,NaN,99.94,NaN,NaN,NaN,NaN,NaN,Rain - Fog
2,2012-04-01 02:00:00,3.8,NaN,1.6,NaN,86.0,NaN,30.0,NaN,26.0,NaN,6.4,NaN,100.00,NaN,NaN,NaN,NaN,NaN,Rain - Fog
3,2012-04-01 03:00:00,2.8,NaN,0.7,NaN,86.0,NaN,29.0,NaN,39.0,NaN,8.0,NaN,100.10,NaN,NaN,NaN,NaN,NaN,Rain - Fog
4,2012-04-01 04:00:00,2.4,NaN,0.4,NaN,87.0,NaN,29.0,NaN,30.0,NaN,9.7,NaN,100.23,NaN,NaN,NaN,NaN,NaN,Moderate Rain


In [9]:
weather.count()

Date/Time              17520
Temp (C)               17498
Temp Flag                  7
Dew Point Temp (C)     17484
Dew Point Temp Flag       21
Rel Hum (%)            17482
Rel Hum Flag              23
Wind Dir (10s deg)     16663
Wind Dir Flag             15
Wind Spd (km/h)        17489
Wind Spd Flag             17
Visibility (km)        17505
Visibility Flag            0
Stn Press (kPa)        17485
Stn Press Flag            20
Hmdx                     663
Hmdx Flag                  0
Wind Chill               548
Wind Chill Flag            0
Weather                13853
dtype: int64

In [10]:
weather.corr()

,Temp (C),Dew Point Temp (C),Rel Hum (%),Wind Dir (10s deg),Wind Spd (km/h),Visibility (km),Visibility Flag,Stn Press (kPa),Hmdx,Hmdx Flag,Wind Chill,Wind Chill Flag
Temp (C),1.000000,0.888455,-0.394187,0.246490,0.069686,0.391891,NaN,-0.224945,0.805810,NaN,0.865309,NaN
Dew Point Temp (C),0.888455,1.000000,0.067536,0.156909,0.010047,0.142343,NaN,-0.272719,0.438942,NaN,0.734499,NaN
Rel Hum (%),-0.394187,0.067536,1.000000,-0.223715,-0.143457,-0.586929,NaN,-0.051888,-0.189728,NaN,0.569483,NaN
Wind Dir (10s deg),0.246490,0.156909,-0.223715,1.000000,0.113131,0.335114,NaN,0.032431,0.158563,NaN,0.105581,NaN
Wind Spd (km/h),0.069686,0.010047,-0.143457,0.113131,1.000000,-0.010544,NaN,-0.293796,-0.042457,NaN,-0.572672,NaN
Visibility (km),0.391891,0.142343,-0.586929,0.335114,-0.010544,1.000000,NaN,0.117370,0.176527,NaN,-0.217380,NaN
Visibility Flag,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Stn Press (kPa),-0.224945,-0.272719,-0.051888,0.032431,-0.293796,0.117370,NaN,1.000000,-0.210623,NaN,-0.039371,NaN
Hmdx,0.805810,0.438942,-0.189728,0.158563,-0.042457,0.176527,NaN,-0.210623,1.000000,NaN,NaN,NaN
Hmdx Flag,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's keep the temperature only !

In [71]:
temp = weather[['Temp (C)', 'Date/Time']].set_index('Date/Time')

In [72]:
temp.tail()

,Temp (C)
Date/Time,
2014-03-31 19:00:00,8.6
2014-03-31 20:00:00,8.2
2014-03-31 21:00:00,7.0
2014-03-31 22:00:00,6.3
2014-03-31 23:00:00,8.1


**There is a discrepancy between the index of weather and the index of power data, this will cause a problem  for th energy kit**

In [69]:
elec_res.head()

,load
date,
2012-04-01 07:00:00,1947.400000
2012-04-01 07:30:00,1269.766667
2012-04-01 08:00:00,1645.300000
2012-04-01 08:30:00,845.666667
2012-04-01 09:00:00,1584.966667


In [77]:
df = elec_res.merge(temp, left_index=True, right_index=True, how='left')
df = df.fillna(method='ffill')

In [78]:
df.head()

,load,Temp (C)
date,,
2012-04-01 07:00:00,1947.400000,4.7
2012-04-01 07:30:00,1269.766667,4.7
2012-04-01 08:00:00,1645.300000,5.3
2012-04-01 08:30:00,845.666667,5.3
2012-04-01 09:00:00,1584.966667,5.9


In [79]:
temp_kit = df.drop('load',1)

In [80]:
temp_kit.index = (temp_kit.index.astype(int)/1e9).astype(int).rename('date')
temp_kit = temp_kit.rename(columns={'Temp (C)':'temp'})

In [81]:
temp_kit.head()

,temp
date,
1333263600,4.7
1333265400,4.7
1333267200,5.3
1333269000,5.3
1333270800,5.9


In [82]:
temp_kit.shape

(35040, 1)

In [88]:
#Burnaby, British Columbia latitude and longitude
latitude = 49.249444
longitude = -122.979722
str_loc = str(latitude)+','+str(longitude)

In [89]:
cache = []
for i,timestamp in enumerate(temp_kit.index):
    data = []
    data.append(str_loc+','+str(timestamp))
    dico = {"temperature":temp_kit.temp[timestamp]}
    data.append(dico)
    cache.append(data)

In [90]:
with open('provider/weather_cache_ampds.json', 'w') as f:
    json.dump(cache, f, indent=2)

In [87]:
temp_kit.to_csv('data/ampds_temp_30T.csv')